In [ ]:
# !pip install -r requirements.txt

In [ ]:
import asyncio
from contextlib import asynccontextmanager

from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv

load_dotenv()

In [ ]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
import os
from pathlib import Path
from flask import Flask

# Define the path to your MCP server
server_params = StdioServerParameters(
    command="docker",
    args=[
        "run",
        "-i",
        "--rm",
        "-e",
        "GITHUB_PERSONAL_ACCESS_TOKEN",
        "ghcr.io/github/github-mcp-server"
      ],
    env={
        "GITHUB_PERSONAL_ACCESS_TOKEN": os.environ.get("GITHUB_PERSONAL_ACCESS_TOKEN"),
    }
)

@asynccontextmanager
async def mcp_tools():
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            tools = await load_mcp_tools(session)
            yield tools


llm = ChatOpenAI(model="gpt-4.1", temperature=0)

async def ai(query: str):
    async with mcp_tools() as tools:
        agent_executor = create_react_agent(llm, tools)

        events = agent_executor.astream(
            {"messages": [("system",
                           "You are a web server, serving HTML web pages. Do not use markdown for formatting. "
                           "All output must be in HTML format and will be displayed to an end user. "
                           "You need to start with doctype and html tags and provide the entire page"),
                          ("user", query)]},
            stream_mode="values",
        )

        async for event in events:
            event["messages"][-1].pretty_print()
            data = event["messages"][-1]

        return data.content


app = Flask(__name__)
    
@app.route('/', methods=['GET'])
def get_data():
    return asyncio.run(ai(Path("my_app-step-3-mcp.txt").read_text()))

app.run(debug=False, port=5001, use_reloader=False)


## Goals
- Write natural language only
- Unstructured data
- Implicit algorithms
- ~~Dynamic tools~~
- ~~Human in the loop built-in~~
- ~~Consistent output~~
- ~~Iterative development~~